In [248]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

pd.options.display.max_columns = None # affiche toutes les colonnes lors de l'affichage des DataFrames
pd.options.display.float_format = '{:.2f}'.format # affiche les nombres à virgule flottante avec 2 décimales

In [336]:
source_file = "../data/1-raw/circulations/slo_annuel_2023.csv"
dest_file = "../data/2-clean/circulations/slo_2023.parquet"

# Ingestion circulation ferroviaire

Notebook de découverte et ingestion des données de circulation ferroviaires.

Chaque colonne du jeu de données est expliquée dans `../data/1-raw/circulations/readme.md`

In [337]:
df_raw = pd.read_csv(
    source_file,
    usecols = ['id_circ', 'num_marche', 'code_ci_origine', 'lib_ci_origine', 'code_ci_destination', 'lib_ci_destination', 'ui', 'lib_ui', 'tct', 'lib_tct', 'rang', 'code_ci_jalon', 'code_ch_jalon', 'lib_ci_jalon', 'distance_cumul', 'type_horaire', 'id_engin', 'mode_traction', 'date_circ', 'dh_the_jalon', 'dh_obs_jalon', 'dh_est_jalon'], 
    dtype={
        "id_circ": "string",
        "num_marche": "string",
        "code_ci_origine": "string",
        "lib_ci_origine": "string",
        "code_ci_destination": "string",
        "lib_ci_destination": "string",
        "ui": "string",
        "lib_ui": "string",
        "tct": "string",
        "lib_tct": "string",
        "rang" : "Int64",
        "code_ci_jalon": "string",
        "code_ch_jalon": "string",
        "lib_ci_jalon": "string",
        "distance_cumul" : "Int64",
        "type_horaire": "string",
        "id_engin": "string",
        "mode_traction": "string"
    },
    parse_dates=[
        "date_circ",
        "dh_the_jalon",
        "dh_obs_jalon",
        "dh_est_jalon"
        ]
    )

In [251]:
df_raw.shape

(1800205, 22)

# copie

In [252]:
df = df_raw.copy()

# Compréhension des données

In [253]:
df.shape

(1800205, 22)

In [254]:
df.head()

,date_circ,id_circ,num_marche,code_ci_origine,lib_ci_origine,code_ci_destination,lib_ci_destination,ui,lib_ui,tct,lib_tct,rang,code_ci_jalon,code_ch_jalon,lib_ci_jalon,distance_cumul,type_horaire,dh_the_jalon,dh_obs_jalon,dh_est_jalon,id_engin,mode_traction
0,2023-01-01,76031871,9568,212076,Strasbourg-Port-du-Rhin,113001,Paris-Est,1187,SNCF-VOYAGES,LVC,"TAGV International France Allemagne, à charge",0,212076,XD,Strasbourg-Port-du-Rhin,0,P,2023-01-01 08:40:01,2023-01-01 08:40:41,2023-01-01 08:40:41,ICE3,E
1,2023-01-01,76031871,9568,212076,Strasbourg-Port-du-Rhin,113001,Paris-Est,1187,SNCF-VOYAGES,LVC,"TAGV International France Allemagne, à charge",15,212027,BV,Strasbourg-Ville,7752,A,2023-01-01 08:47:01,2023-01-01 08:47:16,2023-01-01 08:47:16,ICE3,E
2,2023-01-01,76031871,9568,212076,Strasbourg-Port-du-Rhin,113001,Paris-Est,1187,SNCF-VOYAGES,LVC,"TAGV International France Allemagne, à charge",16,212027,BV,Strasbourg-Ville,7752,D,2023-01-01 08:52:00,2023-01-01 08:52:38,2023-01-01 08:52:38,ICE3,E
3,2023-01-01,76031871,9568,212076,Strasbourg-Port-du-Rhin,113001,Paris-Est,1187,SNCF-VOYAGES,LVC,"TAGV International France Allemagne, à charge",152,113001,00,Paris-Est,447213,A,2023-01-01 10:38:00,2023-01-01 10:35:42,2023-01-01 10:35:42,ICE3,E
4,2023-01-01,76031881,9582,751008,Marseille-St-Charles,212076,Strasbourg-Port-du-Rhin,1187,SNCF-VOYAGES,LVC,"TAGV International France Allemagne, à charge",0,751008,BV,Marseille-St-Charles,0,D,2023-01-01 08:11:01,2023-01-01 08:11:59,2023-01-01 08:11:59,TGV2N2,E


In [255]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800205 entries, 0 to 1800204
Data columns (total 22 columns):
 #   Column               Dtype         
---  ------               -----         
 0   date_circ            datetime64[ns]
 1   id_circ              string        
 2   num_marche           string        
 3   code_ci_origine      string        
 4   lib_ci_origine       string        
 5   code_ci_destination  string        
 6   lib_ci_destination   string        
 7   ui                   string        
 8   lib_ui               string        
 9   tct                  string        
 10  lib_tct              string        
 11  rang                 Int64         
 12  code_ci_jalon        string        
 13  code_ch_jalon        string        
 14  lib_ci_jalon         string        
 15  distance_cumul       Int64         
 16  type_horaire         string        
 17  dh_the_jalon         datetime64[ns]
 18  dh_obs_jalon         datetime64[ns]
 19  dh_est_jalon         

In [256]:
# premier constat : il y a énormément de redondance dans les données, car les infos de chaque train (id_circ, date_circ, code_ci_origine, lib_ci_origine, etc) sont répétées pour chaque point de passage du train, alors même qu'ils ne changent pas.
# une meilleure organisation serait de séparer les informations statiques du train des informations dynamiques liées aux points de passage, dans deux tables distinctes.
# pour simplifier l'analyse, on ne gardera ici que le départ et l'arrivée, en agrégant les données sur une seule ligne par trajet (id_circ).

In [257]:
# df.describe(include='all')

In [258]:
df['id_circ'].nunique() # nombre de circulations uniques (trajets réalisés)

251900

In [259]:
df['num_marche'].nunique() # nombre de numéros de marche uniques (trajets planifiés : gares, horaires)

2940

In [260]:
df['code_ch_jalon'].value_counts() # peu utile dans notre analyse (type de jalon : batiment voyageur, halte, ...)

code_ch_jalon
BV    1481496
00     249901
XB      25985
XT      16101
XS      11224
XD       9308
XL       3930
XI       2260
Name: count, dtype: Int64

In [261]:
df['lib_ui'].value_counts()

lib_ui
SNCF-VOYAGES         1664184
THI FACTORY            50860
OSLO                   32886
EUROSTAR FRANCE        22078
TRENITALIA FRANCE      18348
RENFE VIAJEROS          6970
SNCF-VOYAGEURS          3279
SNCF-INTERCITES         1600
Name: count, dtype: Int64

In [262]:
df['lib_tct'].value_counts()

lib_tct
TAGV - Sud-Est, à charge                                                       453100
TAGV - Atlantique, à charge                                                    338738
TAGV - Nord, à charge                                                          227642
TAGV - Est, à charge                                                           217874
TAGV, axe Bretagne, à charge                                                   105116
TAGV (Train A Grande Vitesse - à charge)                                        97131
TAGV International France Suisse, à charge                                      75713
TAGV haute capacité Sud-Est, à charge                                           56712
TAGV haute capacité Nord, à charge                                              55863
TAGV haute capacité Atlantique, à charge                                        45568
TAGV International France Allemagne, à charge                                   41257
GL Inter-Villes - Train classique, VL>= 160 km

In [263]:
df['mode_traction'].value_counts()

mode_traction
E    1790119
T       2408
B         19
Name: count, dtype: Int64

In [264]:
df.isna().sum()

date_circ                  0
id_circ                    0
num_marche                 0
code_ci_origine            0
lib_ci_origine            62
code_ci_destination        0
lib_ci_destination        26
ui                         0
lib_ui                     0
tct                        0
lib_tct                    0
rang                       0
code_ci_jalon              0
code_ch_jalon              0
lib_ci_jalon           13986
distance_cumul             0
type_horaire               2
dh_the_jalon           37217
dh_obs_jalon           44738
dh_est_jalon              29
id_engin                6796
mode_traction           7659
dtype: int64

In [265]:
# df[df.duplicated()]
# aucun doublon, super

# Nettoyage des données

In [266]:
"""# conversion des dates -> réalisé dès l'ingestion
df['date_circ'] = pd.to_datetime(df['date_circ'], errors='coerce', format='%Y-%m-%d')
df['dh_the_jalon'] = pd.to_datetime(df['dh_the_jalon'], errors='coerce', format='%Y-%m-%d %H:%M:%S')
df['dh_obs_jalon'] = pd.to_datetime(df['dh_obs_jalon'], errors='coerce', format='%Y-%m-%d %H:%M:%S')
df['dh_est_jalon'] = pd.to_datetime(df['dh_est_jalon'], errors='coerce', format='%Y-%m-%d %H:%M:%S')
df.shape"""

"# conversion des dates -> réalisé dès l'ingestion\ndf['date_circ'] = pd.to_datetime(df['date_circ'], errors='coerce', format='%Y-%m-%d')\ndf['dh_the_jalon'] = pd.to_datetime(df['dh_the_jalon'], errors='coerce', format='%Y-%m-%d %H:%M:%S')\ndf['dh_obs_jalon'] = pd.to_datetime(df['dh_obs_jalon'], errors='coerce', format='%Y-%m-%d %H:%M:%S')\ndf['dh_est_jalon'] = pd.to_datetime(df['dh_est_jalon'], errors='coerce', format='%Y-%m-%d %H:%M:%S')\ndf.shape"

In [267]:
"""# Homogénéisation des colonnes d'identifiants -> réalisé dès l'ingestion
df['id_circ'] = df['id_circ'].astype("string")
df['num_marche'] = df['num_marche'].astype("string")
df['code_ci_origine'] = df['code_ci_origine'].astype("string")
df["code_ch_origine"] = df['code_ch_origine'].astype("string")
df['code_ci_destination'] = df['code_ci_destination'].astype("string")
df['code_ci_jalon'] = df['code_ci_jalon'].astype("string")
df['lib_ci_destination'] = df['lib_ci_destination'].astype("string")
""" 

'# Homogénéisation des colonnes d\'identifiants -> réalisé dès l\'ingestion\ndf[\'id_circ\'] = df[\'id_circ\'].astype("string")\ndf[\'num_marche\'] = df[\'num_marche\'].astype("string")\ndf[\'code_ci_origine\'] = df[\'code_ci_origine\'].astype("string")\ndf["code_ch_origine"] = df[\'code_ch_origine\'].astype("string")\ndf[\'code_ci_destination\'] = df[\'code_ci_destination\'].astype("string")\ndf[\'code_ci_jalon\'] = df[\'code_ci_jalon\'].astype("string")\ndf[\'lib_ci_destination\'] = df[\'lib_ci_destination\'].astype("string")\n'

In [268]:
# compte les types dans une colonne donnée -> plus utile car déjà homogénisé dès l'ingestion
# df['lib_ci_destination'].map(type).value_counts()

In [269]:
# affiche les lignes où le type est float
# df[df['lib_ci_destination'].map(type) != str]

In [270]:
df[df['dh_the_jalon'].isna() | df['dh_obs_jalon'].isna() | df['dh_est_jalon'].isna()].shape 
# constat 2024 : 88739 lignes sur 1873489, soit 4.7% de lignes avec une ou plusieurs dates manquantes
# en réalité, si dh_obs_jalon est manquant, on peut compenser par dh_est_jalon.

(81977, 22)

In [271]:
# lignes avec date observée ET estimée manquante : aucune, super, on aura toujours au moins une date pour calcul
df[df['dh_obs_jalon'].isna() & df['dh_est_jalon'].isna()].shape

(0, 22)

In [272]:
# pas d'horaire théorique : impossible de calculer le retard du train.
df[df['dh_the_jalon'].isna()].shape
# constat 2024 : 44380 sur 1873489, soit 2.3% de lignes concernées.
# On peut drop ces lignes sans risque.

(37217, 22)

In [273]:
df = df.dropna(subset=['dh_the_jalon'])
df.shape

(1762988, 22)

In [290]:
# En fait, on pourrait même drop directement toutes les lignes qui contiennent des valeurs manquantes (peu importe la colonne),
# car elles représentent peu de données et simplifieraient l'analyse ensuite.
df = df.dropna()
df.shape

(1703382, 24)

In [333]:
df[df.isna().any(axis=1)]

,date_circ,id_circ,num_marche,code_ci_origine,lib_ci_origine,code_ci_destination,lib_ci_destination,ui,lib_ui,tct,lib_tct,rang,code_ci_jalon,code_ch_jalon,lib_ci_jalon,distance_cumul,type_horaire,dh_the_jalon,dh_obs_jalon,dh_est_jalon,id_engin,mode_traction,is_depart,is_arrivee


In [291]:
# Perte de données à ce stade : 
lignes_initiales = df_raw.shape[0]
lignes_actuelles = df.shape[0]
print(f'Jalons : {lignes_actuelles} sur {lignes_initiales} :  {(lignes_actuelles - lignes_initiales) / lignes_initiales:.2%} de pertes')
trajets_actuels = df['id_circ'].nunique()
trajets_initiaux = df_raw['id_circ'].nunique()
print(f"Trajets : {trajets_actuels} sur {trajets_initiaux} : {(trajets_actuels - trajets_initiaux) / trajets_initiaux:.2%} de pertes")
# En réalité, c'est plus compliqué que ça... car il y a un grand nombre de jalons qui ne nous serviront pas du tout (les arrêts intermédiaires), et de trajets potentiellement incomplets dans le jeu de données intial. Mais ça, les chiffres ci dessus ne le reflètent pas.
# Cela étant dit, 5% c'est tout à fait raisonnable, et ça simplifie l'analyse ensuite.

Jalons : 1703382 sur 1800205 :  -5.38% de pertes
Trajets : 249984 sur 251900 : -0.76% de pertes


# Pivot : une ligne par trajet

In [292]:
# on flag le départ et l'arrivée : 
# code_ci_jalon == code_ci_origine & type_horaire = 'D' ou 'P' ou 'I' -> départ 
# code_ci_jalon == code_ci_destination & type_horaire = 'A' ou 'P' ou 'I' -> arrivée 
# ('D' : départ, 'A' : arrivée, 'P' : passage)
# ('I' : non documenté, concerne très peu de lignes. gare intermédiaire, peut-être ?)
# En situation pro, il aurait fallu enquêter là dessus pour s'assurer que mes filtres sont pertinents.
df['is_depart'] = (df['code_ci_jalon'] == df['code_ci_origine']) & (df['type_horaire'].isin(['D', 'P', 'I']))
df['is_arrivee'] = (df['code_ci_jalon'] == df['code_ci_destination']) & (df['type_horaire'].isin(['A', 'P', 'I']))
df.shape

(1703382, 24)

In [318]:
# on garde UNIQUEMENT les lignes de DEPART et d'ARRIVEE
df_trajets = df[df['is_depart'] | df['is_arrivee']].copy()
df_trajets.shape

(497212, 24)

In [319]:
# trajets qui ont is_depart et is_arrivee true en même temps ?
df_trajets[df_trajets['is_depart'] & df_trajets['is_arrivee']]
# constat 2024 : il y a 74 lignes, n'apporte aucune info pertinente (trajet Modane -> Modane par exemple)
df_trajets = df_trajets[~(df_trajets['is_depart'] & df_trajets['is_arrivee'])]
df_trajets.shape

(497209, 24)

In [320]:
df_trajets['is_depart'].value_counts()

is_depart
False    248706
True     248503
Name: count, dtype: Int64

In [321]:
df_trajets['is_arrivee'].value_counts()

is_arrivee
True     248706
False    248503
Name: count, dtype: Int64

In [322]:
# remplace is_depart et is_arrivee par une seule colonne : type_arret
df_trajets['type_arret'] = np.where(df_trajets['is_depart'], 'depart', 'arrivee') # on peut faire ça car on a déjà filtré les lignes où les deux sont true
df_trajets = df_trajets.drop(columns=['is_depart', 'is_arrivee'])
df_trajets['type_arret'].value_counts()

type_arret
arrivee    248706
depart     248503
Name: count, dtype: int64

In [323]:
# df_trajets[df_trajets['id_circ'] == '82520670']
# constat 2024 : LIB_TCT peut changer en cours de route (exemple : TAGV Atlantique -> TAGV Bretagne).
# à passer en valeur agrégée dans le pivot plus bas.

In [324]:
# df_trajets[df_trajets['id_circ'] == '84942965']
# constat 2024 : ID_ENGIN peut changer en cours de route, dans de très rares cas (56 lignes, donc 28 trajets).
# à passer en valeur agrégée dans le pivot plus bas.

In [325]:
# on vérifie qu'il n'y a aucun doublon pour la clef utilisée dans le pivot plus bas
df_trajets[df_trajets.duplicated(subset=['id_circ', 'date_circ', 'num_marche', 'code_ci_origine', 'lib_ci_origine', 
                                         'code_ci_destination', 'lib_ci_destination', 'lib_ui', 'id_engin', 'type_arret'], keep=False)].head()

,date_circ,id_circ,num_marche,code_ci_origine,lib_ci_origine,code_ci_destination,lib_ci_destination,ui,lib_ui,tct,lib_tct,rang,code_ci_jalon,code_ch_jalon,lib_ci_jalon,distance_cumul,type_horaire,dh_the_jalon,dh_obs_jalon,dh_est_jalon,id_engin,mode_traction,type_arret
132318,2023-01-01,76031822,9240,742007,Modane,686006,Paris-Gare-de-Lyon,1187,SNCF-VOYAGES,LVR,"TAGV International France Italie, à charge",0,742007,XI,Modane,0,P,2023-01-01 08:51:01,2023-01-01 08:47:00,2023-01-01 08:47:00,TGVR,E,depart
132320,2023-01-01,76031822,9240,742007,Modane,686006,Paris-Gare-de-Lyon,1187,SNCF-VOYAGES,LVR,"TAGV International France Italie, à charge",3,742007,BV,Modane,11360,D,2023-01-01 09:13:00,2023-01-01 09:13:00,2023-01-01 09:13:00,TGVR,E,depart
132335,2023-01-01,76033080,9241,686006,Paris-Gare-de-Lyon,742007,Modane,1187,SNCF-VOYAGES,LVR,"TAGV International France Italie, à charge",200,742007,BV,Modane,630214,A,2023-01-01 10:51:00,2023-01-01 10:51:00,2023-01-01 10:51:00,TGVR,E,arrivee
132337,2023-01-01,76033080,9241,686006,Paris-Gare-de-Lyon,742007,Modane,1187,SNCF-VOYAGES,LVR,"TAGV International France Italie, à charge",203,742007,XI,Modane,641574,A,2023-01-01 11:09:00,2023-01-01 11:09:00,2023-01-01 11:09:00,TGVR,E,arrivee
132338,2023-01-01,76046441,9244,742007,Modane,741009,Chambéry-Challes-les-Eaux,1187,SNCF-VOYAGES,LVR,"TAGV International France Italie, à charge",0,742007,XI,Modane,0,P,2023-01-01 14:51:01,2023-01-01 14:51:00,2023-01-01 14:51:00,TGVR,E,depart


In [326]:
# constat 2024 : 2451 trajets avec des départs ou arrivées en double.
# Idéalement, on devrait enquêter pour comprendre la raison derrière. 
# Ici, on constate que pour à peu près une ligne sur deux, il manque des dates (théorique, observée ou estimée).
# On va donc garder les lignes avec le plus d'informations (non nulles) sur les dates :
df_trajets['num_dates_non_nulles'] = df_trajets[['dh_the_jalon', 'dh_obs_jalon', 'dh_est_jalon']].notna().sum(axis=1) # compte le nombre de dates non nulles par ligne
df_trajets = df_trajets.sort_values(by=['id_circ','type_arret', 'num_dates_non_nulles'], ascending=[True, True, False])
df_trajets = df_trajets.drop_duplicates(subset=['id_circ', 'date_circ', 'num_marche', 'code_ci_origine', 'lib_ci_origine', 
            'code_ci_destination', 'lib_ci_destination', 'lib_ui', 'id_engin', 'type_arret'], keep='first') # garde la ligne avec le plus de dates non nulles

In [327]:
# vérification qu'il n'y a plus aucun doublon
df_trajets[df_trajets.duplicated(subset=['id_circ', 'date_circ', 'num_marche', 'code_ci_origine', 'lib_ci_origine', 
            'code_ci_destination', 'lib_ci_destination', 'lib_ui', 'id_engin', 'type_arret'], keep=False)]

,date_circ,id_circ,num_marche,code_ci_origine,lib_ci_origine,code_ci_destination,lib_ci_destination,ui,lib_ui,tct,lib_tct,rang,code_ci_jalon,code_ch_jalon,lib_ci_jalon,distance_cumul,type_horaire,dh_the_jalon,dh_obs_jalon,dh_est_jalon,id_engin,mode_traction,type_arret,num_dates_non_nulles


In [328]:
# compte le nombre de types d'arrêt par id_circ (il en faut 1 ou 2 : départ et arrivée)
df_trajets.groupby('id_circ')['type_arret'].nunique().value_counts()
# constat 2024 : on a 4565 trajets sur 255267 pour lesquels il manque le départ ou l'arrivée, soit 1.8%. supprimé plus loin, après le pivot.

type_arret
2    245044
1      4913
Name: count, dtype: int64

In [329]:
### PIVOT : pour avoir UNE LIGNE PAR TRAJET
df_trajets_pivot = df_trajets.pivot_table(
    index = ['id_circ', 'date_circ', 'num_marche', 'code_ci_origine', 'lib_ci_origine', 
            'code_ci_destination', 'lib_ci_destination', 'lib_ui'],
    columns = ['type_arret'],
    values = ['dh_the_jalon', 'dh_est_jalon', 'dh_obs_jalon', 'distance_cumul', 'lib_tct', 'id_engin', 'code_ch_jalon'],
    aggfunc = 'first'  # on a déjà géré les doublons, donc ce paramètre ne devrait avoir aucun impact.
)

# on aplatit les colonnes multi-index
df_trajets_pivot.columns = ['_'.join(col).strip() for col in df_trajets_pivot.columns.values]
df_trajets_pivot = df_trajets_pivot.reset_index()

# on renomme les colonnes
df_trajets_pivot = df_trajets_pivot.rename(columns={
    'dh_the_jalon_depart': 'depart_theorique',
    'dh_the_jalon_arrivee': 'arrivee_theorique',
    'dh_est_jalon_depart': 'depart_estime',
    'dh_est_jalon_arrivee': 'arrivee_estime',
    'dh_obs_jalon_depart': 'depart_observe',
    'dh_obs_jalon_arrivee': 'arrivee_observe',
    'distance_cumul_arrivee': 'distance_totale',
    'code_ch_jalon_depart': 'code_ch_origine',
    'code_ch_jalon_arrivee': 'code_ch_destination',
})

# on réorganise l'ordre des colonnes
df_trajets_pivot = df_trajets_pivot[[
    'id_circ', 'date_circ', 'num_marche', 
    'code_ci_origine', 'lib_ci_origine', 'code_ch_origine',
    'code_ci_destination', 'lib_ci_destination', 'code_ch_destination',
    'lib_ui', 'lib_tct_depart', 'lib_tct_arrivee', 
    'id_engin_depart', 'id_engin_arrivee',
    'depart_theorique', 'depart_observe', 'depart_estime',
    'arrivee_theorique', 'arrivee_observe', 'arrivee_estime', 
    'distance_totale'
]]

df_trajets_pivot

,id_circ,date_circ,num_marche,code_ci_origine,lib_ci_origine,code_ch_origine,code_ci_destination,lib_ci_destination,code_ch_destination,lib_ui,lib_tct_depart,lib_tct_arrivee,id_engin_depart,id_engin_arrivee,depart_theorique,depart_observe,depart_estime,arrivee_theorique,arrivee_observe,arrivee_estime,distance_totale
0,76027713,2023-01-01,2352,182014,Colmar,BV,212027,Strasbourg-Ville,BV,SNCF-VOYAGES,"TAGV - Est, à charge","TAGV - Est, à charge",TGV2N2,TGV2N2,2023-01-01 07:37:01,2023-01-01 07:36:39,2023-01-01 07:36:39,2023-01-01 08:09:00,2023-01-01 08:06:28,2023-01-01 08:06:28,65817
1,76027729,2023-01-01,2579,141002,Nancy-Ville,BV,113001,Paris-Est,00,SNCF-VOYAGES,"TAGV - Est, à charge","TAGV - Est, à charge",TGVR,TGVR,2023-01-01 10:11:01,2023-01-01 10:11:16,2023-01-01 10:11:16,2023-01-01 11:50:00,2023-01-01 11:48:18,2023-01-01 11:48:18,327322
2,76027732,2023-01-01,2587,113001,Paris-Est,00,212027,Strasbourg-Ville,BV,SNCF-VOYAGES,"TAGV - Est, à charge","TAGV - Est, à charge",TGVR,TGVR,2023-01-01 09:13:01,2023-01-01 09:44:44,2023-01-01 09:44:44,2023-01-01 12:22:00,2023-01-01 12:39:04,2023-01-01 12:39:04,476867
3,76027758,2023-01-01,2407,113001,Paris-Est,00,212027,Strasbourg-Ville,BV,SNCF-VOYAGES,"TAGV - Est, à charge","TAGV - Est, à charge",TGVR,TGVR,2023-01-01 07:58:01,2023-01-01 07:57:42,2023-01-01 07:57:42,2023-01-01 09:59:00,2023-01-01 09:58:02,2023-01-01 09:58:02,439461
4,76027764,2023-01-01,2535,141002,Nancy-Ville,BV,113001,Paris-Est,00,SNCF-VOYAGES,"TAGV - Est, à charge","TAGV - Est, à charge",TGVR,TGVR,2023-01-01 08:16:01,2023-01-01 08:17:49,2023-01-01 08:17:49,2023-01-01 09:46:01,2023-01-01 09:47:54,2023-01-01 09:47:54,327322
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249952,82525879,2023-12-30,8480,581009,Bordeaux-St-Jean,BV,391003,Paris-Montparnasse,BV,SNCF-VOYAGES,"TAGV - Atlantique, à charge","TAGV - Atlantique, à charge",TGVAT,TGVAT,2023-12-30 05:18:00,2023-12-30 05:45:01,2023-12-30 05:45:01,2023-12-30 08:34:30,2023-12-30 08:40:42,2023-12-30 08:40:42,547342
249953,82526799,2023-12-30,7534,317586,Boulogne-Ville,<NA>,271007,Paris-Nord,BV,SNCF-VOYAGES,<NA>,"TAGV - Nord, à charge",<NA>,TGV-D,NaT,NaT,NaT,2023-12-30 19:09:00,2023-12-30 19:08:56,2023-12-30 19:08:56,369618
249954,82542771,2023-12-31,7611,391003,Paris-Montparnasse,BV,474007,Brest,BV,SNCF-VOYAGES,"TAGV haute capacité Atlantique, à charge","TAGV haute capacité Atlantique, à charge",TGVDAS,TGVDAS,2023-12-31 06:48:00,2023-12-31 06:44:27,2023-12-31 06:44:27,2023-12-31 10:34:00,2023-12-31 10:31:00,2023-12-31 10:31:00,613002
249955,82543804,2023-12-31,7534,317586,Boulogne-Ville,<NA>,271007,Paris-Nord,BV,SNCF-VOYAGES,<NA>,"TAGV - Nord, à charge",<NA>,TGV-D,NaT,NaT,NaT,2023-12-31 19:09:00,2023-12-31 19:09:12,2023-12-31 19:09:12,369618


In [330]:
# id_circ en doublon ? -> traités plus haut.
df_trajets_pivot[df_trajets_pivot.duplicated(subset=['id_circ'], keep=False)].shape

(0, 21)

In [ ]:
# lignes avec valeurs manquantes :
df_trajets_pivot[df_trajets_pivot.isna().any(axis=1)].shape
# seulement 5758 sur 259764, soit 2.2% de lignes concernées. on peut supprimer

# nb : c'est normal qu'il y ait des valeurs manquantes, alors même que l'on avait supprimé tous les na dans le df initial.
#  En effet, certains trajets n'ont pas de départ ou d'arrivée (ex : trajets partiels dans le jeu de données initial). 

(4913, 21)

In [307]:
df_trajets_pivot = df_trajets_pivot.dropna()
df_trajets_pivot.shape
# on pourrait éventuellement faire un nettoyage plus fin, en gardant les lignes où il manque seulement l'horaire observé, et calculer à partir de l'estimé.
# mais comme le volume de données est très faible, ça n'en vaut pas l'effort.

(245044, 21)

In [308]:
# vérification qu'il n'y a aucun missing value dans les horaires observés
df_trajets_pivot[df_trajets_pivot['depart_observe'].isna() | df_trajets_pivot['arrivee_observe'].isna()].shape

(0, 21)

In [309]:
# trajets avec date arrivée < date départ : aucun, super
df_trajets_pivot[df_trajets_pivot['arrivee_observe'] < df_trajets_pivot['depart_observe']]

,id_circ,date_circ,num_marche,code_ci_origine,lib_ci_origine,code_ch_origine,code_ci_destination,lib_ci_destination,code_ch_destination,lib_ui,lib_tct_depart,lib_tct_arrivee,id_engin_depart,id_engin_arrivee,depart_theorique,depart_observe,depart_estime,arrivee_theorique,arrivee_observe,arrivee_estime,distance_totale


In [310]:
# taux de perte de trajets après nettoyage
n_trajets_final = df_trajets_pivot.shape[0]
n_trajets_initial = df_raw['id_circ'].nunique()
perte_relative = (n_trajets_initial - n_trajets_final) / n_trajets_initial
print(f"Après nettoyage, on a {n_trajets_final:_} trajets sur {n_trajets_initial:_} initialement, soit une perte de {perte_relative:.2%} des trajets.")

Après nettoyage, on a 245_044 trajets sur 251_900 initialement, soit une perte de 2.72% des trajets.


# Calcul des colonnes dérivées

Pour contexte, la SNCF compte les retards comme suit : 
- Un train est considéré à l'heure si son retard au terminus est inférieur à 5min pour un parcours inférieur à 1h30
- Un train est considéré à l'heure si son retard au terminus est inférieur à 10min pour un parcours entre 1h30 et 3h
- Un train est considéré à l'heure si son retard au terminus est inférieur à 15min pour un parcours au-delà de 3h

Tous ces retards ne sont donc pas comptabilisés par la SNCF.
Nous allons utiliser des catégories similaires, pour pouvoir comparer avec les chiffres officiels.

In [311]:
df_retards = df_trajets_pivot.copy()
df_retards.head()

,id_circ,date_circ,num_marche,code_ci_origine,lib_ci_origine,code_ch_origine,code_ci_destination,lib_ci_destination,code_ch_destination,lib_ui,lib_tct_depart,lib_tct_arrivee,id_engin_depart,id_engin_arrivee,depart_theorique,depart_observe,depart_estime,arrivee_theorique,arrivee_observe,arrivee_estime,distance_totale
0,76027713,2023-01-01,2352,182014,Colmar,BV,212027,Strasbourg-Ville,BV,SNCF-VOYAGES,"TAGV - Est, à charge","TAGV - Est, à charge",TGV2N2,TGV2N2,2023-01-01 07:37:01,2023-01-01 07:36:39,2023-01-01 07:36:39,2023-01-01 08:09:00,2023-01-01 08:06:28,2023-01-01 08:06:28,65817
1,76027729,2023-01-01,2579,141002,Nancy-Ville,BV,113001,Paris-Est,00,SNCF-VOYAGES,"TAGV - Est, à charge","TAGV - Est, à charge",TGVR,TGVR,2023-01-01 10:11:01,2023-01-01 10:11:16,2023-01-01 10:11:16,2023-01-01 11:50:00,2023-01-01 11:48:18,2023-01-01 11:48:18,327322
2,76027732,2023-01-01,2587,113001,Paris-Est,00,212027,Strasbourg-Ville,BV,SNCF-VOYAGES,"TAGV - Est, à charge","TAGV - Est, à charge",TGVR,TGVR,2023-01-01 09:13:01,2023-01-01 09:44:44,2023-01-01 09:44:44,2023-01-01 12:22:00,2023-01-01 12:39:04,2023-01-01 12:39:04,476867
3,76027758,2023-01-01,2407,113001,Paris-Est,00,212027,Strasbourg-Ville,BV,SNCF-VOYAGES,"TAGV - Est, à charge","TAGV - Est, à charge",TGVR,TGVR,2023-01-01 07:58:01,2023-01-01 07:57:42,2023-01-01 07:57:42,2023-01-01 09:59:00,2023-01-01 09:58:02,2023-01-01 09:58:02,439461
4,76027764,2023-01-01,2535,141002,Nancy-Ville,BV,113001,Paris-Est,00,SNCF-VOYAGES,"TAGV - Est, à charge","TAGV - Est, à charge",TGVR,TGVR,2023-01-01 08:16:01,2023-01-01 08:17:49,2023-01-01 08:17:49,2023-01-01 09:46:01,2023-01-01 09:47:54,2023-01-01 09:47:54,327322


In [312]:
# nombre de trajets où il manque des informations temporelles : aucune, puisqu'on a drop les lignes avec des valeurs manquantes plus haut.
# on va donc pouvoir utiliser uniquement les horaires observés (et non estimés) pour le calcul des retards
df_retards[df_retards[['depart_theorique', 'depart_observe', 'depart_estime', 'arrivee_theorique', 'arrivee_observe', 'arrivee_estime']].isna().any(axis=1)].shape

(0, 21)

In [313]:
# on peut donc au passage drop les colonnes d'horaires estimés
df_retards = df_retards.drop(columns=['depart_estime', 'arrivee_estime'])

In [314]:
# NB : un retard négatif indique une avance.

# Retard au départ et à l'arrivée (en minutes)
df_retards['ret_depart'] = ((df_retards['depart_observe'] - df_retards['depart_theorique']).dt.total_seconds()) / 60
df_retards['ret_arrivee'] = ((df_retards['arrivee_observe'] - df_retards['arrivee_theorique']).dt.total_seconds()) / 60

# Catégories de retard à l'arrivée : 'True' si le train a plus de 'n' minutes de retard
for n in [5, 10, 15, 30, 60]:
    df_retards[f"ret_arrivee_{n}min"] = df_retards["ret_arrivee"] >= n

# Catégorie de retard à l'arrivée : finalement remplacé par les catégories ci dessus.
# bins = [float("-inf"), -10, -5, 0, 5, 10, 15, 30, 60, float("inf")]
# labels = ["-10-", "[-10,-5[", "[-5,0[", "[0,5[", "[5,10[", "[10,15[", "[15,30[", "[30,60[", "[60+]"]
# df_retards["ret_arrivee_cat"] = pd.cut(df_retards["ret_arrivee"], bins=bins, labels=labels, right=False)

# Durée théorique du trajet (en minutes)
df_retards['duree_theorique'] = (df_retards['arrivee_theorique'] - df_retards['depart_theorique']).dt.total_seconds() / 60

# Durée théorique du trajet (catégories)
bins = [0, 90, 180, float("inf")]
labels = ["<1h30", "1h30-3h", ">3h"]
df_retards["duree_theorique_cat"] = pd.cut(df_retards["duree_theorique"], bins=bins, labels=labels, right=False)

# Durée observée du trajet (en minutes)
df_retards['duree_observee'] = (df_retards['arrivee_observe'] - df_retards['depart_observe']).dt.total_seconds() / 60

# Mois (1 = janvier, 12 = décembre)
df_retards['mois'] = df_retards['date_circ'].dt.month

# Numéro de semaine dans l'année
df_retards['num_semaine'] = df_retards['date_circ'].dt.isocalendar().week

# Jour de la semaine (0 = lundi, 6 = dimanche)
df_retards['jour_semaine'] = df_retards['date_circ'].dt.dayofweek

# Plage horaire de départ et d'arrivée, par tranche de 1 heure
df_retards['heure_depart'] = df_retards['depart_observe'].dt.hour
df_retards['heure_arrivee'] = df_retards['arrivee_observe'].dt.hour

df_retards

,id_circ,date_circ,num_marche,code_ci_origine,lib_ci_origine,code_ch_origine,code_ci_destination,lib_ci_destination,code_ch_destination,lib_ui,lib_tct_depart,lib_tct_arrivee,id_engin_depart,id_engin_arrivee,depart_theorique,depart_observe,arrivee_theorique,arrivee_observe,distance_totale,ret_depart,ret_arrivee,ret_arrivee_5min,ret_arrivee_10min,ret_arrivee_15min,ret_arrivee_30min,ret_arrivee_60min,duree_theorique,duree_theorique_cat,duree_observee,mois,num_semaine,jour_semaine,heure_depart,heure_arrivee
0,76027713,2023-01-01,2352,182014,Colmar,BV,212027,Strasbourg-Ville,BV,SNCF-VOYAGES,"TAGV - Est, à charge","TAGV - Est, à charge",TGV2N2,TGV2N2,2023-01-01 07:37:01,2023-01-01 07:36:39,2023-01-01 08:09:00,2023-01-01 08:06:28,65817,-0.37,-2.53,False,False,False,False,False,31.98,<1h30,29.82,1,52,6,7,8
1,76027729,2023-01-01,2579,141002,Nancy-Ville,BV,113001,Paris-Est,00,SNCF-VOYAGES,"TAGV - Est, à charge","TAGV - Est, à charge",TGVR,TGVR,2023-01-01 10:11:01,2023-01-01 10:11:16,2023-01-01 11:50:00,2023-01-01 11:48:18,327322,0.25,-1.70,False,False,False,False,False,98.98,1h30-3h,97.03,1,52,6,10,11
2,76027732,2023-01-01,2587,113001,Paris-Est,00,212027,Strasbourg-Ville,BV,SNCF-VOYAGES,"TAGV - Est, à charge","TAGV - Est, à charge",TGVR,TGVR,2023-01-01 09:13:01,2023-01-01 09:44:44,2023-01-01 12:22:00,2023-01-01 12:39:04,476867,31.72,17.07,True,True,True,False,False,188.98,>3h,174.33,1,52,6,9,12
3,76027758,2023-01-01,2407,113001,Paris-Est,00,212027,Strasbourg-Ville,BV,SNCF-VOYAGES,"TAGV - Est, à charge","TAGV - Est, à charge",TGVR,TGVR,2023-01-01 07:58:01,2023-01-01 07:57:42,2023-01-01 09:59:00,2023-01-01 09:58:02,439461,-0.32,-0.97,False,False,False,False,False,120.98,1h30-3h,120.33,1,52,6,7,9
4,76027764,2023-01-01,2535,141002,Nancy-Ville,BV,113001,Paris-Est,00,SNCF-VOYAGES,"TAGV - Est, à charge","TAGV - Est, à charge",TGVR,TGVR,2023-01-01 08:16:01,2023-01-01 08:17:49,2023-01-01 09:46:01,2023-01-01 09:47:54,327322,1.80,1.88,False,False,False,False,False,90.00,1h30-3h,90.08,1,52,6,8,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249950,82524468,2023-12-31,9896,751008,Marseille-St-Charles,BV,191973,Zoufftgen (IE),XL,SNCF-VOYAGES,"TAGV - Est, à charge","TAGV - Est, à charge",TGV2N2,TGV2N2,2023-12-31 15:56:00,2023-12-31 15:55:59,2023-12-31 23:36:00,2024-01-01 00:39:12,1025894,-0.02,63.20,True,True,True,True,True,460.00,>3h,523.22,12,52,6,15,0
249951,82524484,2023-12-31,9977,111849,Marne-la-Vallée-Chessy,BV,281188,Wannehain,XB,THI FACTORY,TAGV (Train A Grande Vitesse - à charge),TAGV (Train A Grande Vitesse - à charge),TGVR,TGVR,2023-12-31 18:42:00,2023-12-31 18:42:11,2023-12-31 19:47:00,2023-12-31 19:47:54,227200,0.18,0.90,False,False,False,False,False,65.00,<1h30,65.72,12,52,6,18,19
249952,82525879,2023-12-30,8480,581009,Bordeaux-St-Jean,BV,391003,Paris-Montparnasse,BV,SNCF-VOYAGES,"TAGV - Atlantique, à charge","TAGV - Atlantique, à charge",TGVAT,TGVAT,2023-12-30 05:18:00,2023-12-30 05:45:01,2023-12-30 08:34:30,2023-12-30 08:40:42,547342,27.02,6.20,True,False,False,False,False,196.50,>3h,175.68,12,52,5,5,8
249954,82542771,2023-12-31,7611,391003,Paris-Montparnasse,BV,474007,Brest,BV,SNCF-VOYAGES,"TAGV haute capacité Atlantique, à charge","TAGV haute capacité Atlantique, à charge",TGVDAS,TGVDAS,2023-12-31 06:48:00,2023-12-31 06:44:27,2023-12-31 10:34:00,2023-12-31 10:31:00,613002,-3.55,-3.00,False,False,False,False,False,226.00,>3h,226.55,12,52,6,6,10


# Export en Parquet

In [315]:
df_retards.to_parquet(dest_file, index=False)

In [316]:
# test d'import
# df_test = pd.read_parquet("../data/2-clean/slo_2024.parquet")
# df_test.head(3)

In [317]:
# Comparaison avec le format CSV :

# écriture + lecture : 9s en CSV contre 1.2s en parquet
# taille du fichier : 90Mo en CSV contre 17Mo en parquet

"""
df_retards.to_csv("../data/2-clean/slo_2024.csv", index=False)
df_test_csv = pd.read_csv("../data/2-clean/slo_2024.csv", parse_dates=[
    "date_circ", "depart_theorique", "depart_observe", "depart_estime",
    "arrivee_theorique", "arrivee_observe", "arrivee_estime"
])
"""


'\ndf_retards.to_csv("../data/2-clean/slo_2024.csv", index=False)\ndf_test_csv = pd.read_csv("../data/2-clean/slo_2024.csv", parse_dates=[\n    "date_circ", "depart_theorique", "depart_observe", "depart_estime",\n    "arrivee_theorique", "arrivee_observe", "arrivee_estime"\n])\n'